In [2]:
import requests
from bs4 import BeautifulSoup
import subprocess
import re
import os
import stanfordnlp
import pandas as pd

from ProcessArticle import ProcessArticle

from WikiArticleXmlHandler import WikiArticleXmlHandler
from WikiDataXmlHandler import WikiDataXmlHandler
import xml.sax
import mwparserfromhell

from timeit import default_timer as timer

from multiprocessing import Pool 
from multiprocessing import Lock
import tqdm 

# List of lists to single list
from itertools import chain

# Sending keyword arguments in map
from functools import partial

import gc

import warnings
warnings.filterwarnings('ignore')

In [3]:
infoboxes = pd.read_csv("./2.data/infobox-list.csv")["infobox template name"].values.tolist()

article_folder = "./0.articles"
articles = [article_folder + "/" + x for x in os.listdir(article_folder)]
article = articles[0] # process only one file for now

wikidata_folder = "./1.wikidata"
wikidatas = [wikidata_folder + "/" + x for x in os.listdir(wikidata_folder)]
wikidata = wikidatas[0]

out_file = "./saved/n_out_nlp_"

nlp = stanfordnlp.Pipeline(models_dir="./stanfordnlp_resources/", use_gpu=False);

func = ProcessArticle(infoboxes, nlp)

Use device: cpu
---
Loading: tokenize
With settings:
{'model_path': './stanfordnlp_resources/en_ewt_models\\en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings:
{'model_path': './stanfordnlp_resources/en_ewt_models\\en_ewt_tagger.pt', 'pretrain_path': './stanfordnlp_resources/en_ewt_models\\en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings:
{'model_path': './stanfordnlp_resources/en_ewt_models\\en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings:
{'model_path': './stanfordnlp_resources/en_ewt_models\\en_ewt_parser.pt', 'pretrain_path': './stanfordnlp_resources/en_ewt_models\\en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': '

In [4]:
if not os.path.exists(out_file + "_wikidata.csv"):
    pd.DataFrame(columns=["title", "label", "type"]).to_csv(
        out_file + "_wikidata.csv", header=True, index=False, sep="\t")

In [5]:
%%time

handler = WikiDataXmlHandler(out_file + "_wikidata.csv")
parser = xml.sax.make_parser()
parser.setContentHandler(handler)

print("Started processing: {}".format(wikidata))
for line in subprocess.Popen(["bzcat"],
                        stdin=open(wikidata),
                        stdout=subprocess.PIPE).stdout:
    try:
        parser.feed(line)
    except StopIteration:
        break

# making sure that everything was written to the file
handler.write_df()

Q237048
No english (en) available for Q237052
No english (en) available for Q237059
No english (en) available for Q237062
No english (en) available for Q237069
No english (en) available for Q237079
No english (en) available for Q237091
No english (en) available for Q237094
No english (en) available for Q237097
No english (en) available for Q237101
No type key in: Q237118
No english (en) available for Q237137
No english (en) available for Q237153
No english (en) available for Q237158
No english (en) available for Q237163
No english (en) available for Q237172
No english (en) available for Q237176
No english (en) available for Q237180
No english (en) available for Q237188
No english (en) available for Q237192
No english (en) available for Q237198
No english (en) available for Q237243
No english (en) available for Q237262
No type key in: Q237274
No english (en) available for Q237296
No english (en) available for Q237297
No english (en) available for Q237300
No english (en) available for Q2

In [ ]:
%%time

handler = WikiArticleXmlHandler(out_file + ".csv", func, 5000)
parser = xml.sax.make_parser()
parser.setContentHandler(handler)

print("Started processing: {}".format(article))
for line in subprocess.Popen(["bzcat"],
                        stdin=open(article),
                        stdout=subprocess.PIPE).stdout:
    try:
        parser.feed(line)
    except StopIteration:
        break

# making sure that everything was written to the file
handler.write_df()